In [1]:
using ArenaSim
using Statistics

┌ Info: Precompiling ArenaSim [09fce650-8ef9-11e8-1aa4-31c92d7f482c]
└ @ Base loading.jl:1187


In [2]:
deckfolder = joinpath(homedir(), "packages/ArenaSim/decks/teferi_control_example")
mtgadecks = readdir(deckfolder)
mtgadecks = map(fn -> ArenaSim.deckreader_mtga_format(joinpath(deckfolder, fn)), mtgadecks)
mtgadecks = ArenaSim.deckinfo.(mtgadecks);

In [3]:
for card in mtgadecks[1]
    println( (card.name, card.amount))
end

("Torrential Gearhulk", 1)
("Fumigate", 2)
("Glimmer of Genius", 2)
("Negate", 1)
("Settle the Wreckage", 3)
("Essence Scatter", 2)
("Disallow", 4)
("Syncopate", 2)
("Blink of an Eye", 1)
("Commit // Memory", 1)
("Hieroglyphic Illumination", 2)
("Search for Azcanta // Azcanta, the Sunken Ruin", 2)
("Cast Out", 3)
("Seal Away", 4)
("Glacial Fortress", 4)
("Irrigated Farmland", 4)
("Meandering River", 2)
("Field of Ruin", 2)
("Arch of Orazca", 1)
("Teferi, Hero of Dominaria", 4)
("Negate", 2)
("Fumigate", 1)
("Torrential Gearhulk", 2)
("Kefnet the Mindful", 1)
("Nezahal, Primal Tide", 1)
("Lyra Dawnbringer", 3)
("Forsake the Worldly", 2)
("Jace's Defeat", 1)
("Sorcerous Spyglass", 1)
("Aether Meltdown", 1)


In [4]:
function summarize_results(packs_opened)
    display(Dict( (k, mean(vec(v))) for (k,v) in packs_opened))
    display("Total (10%, mean, 90%):")
    display((x -> (quantile(vec(x), .1), mean(vec(x)), quantile(vec(x),.9)))( reduce(+, values(packs_opened)) ))
end

summarize_results (generic function with 1 method)

# Account starts with literally nothing (inaccurate)

In [5]:
res = simulate(10000, ArenaSim.deckinfo.(mtgadecks);
    parameters = ArenaSim.SimParameters(welcome_bundle = false,
        kaladesh_grant = false, starter_cards = zeros(Int, length(ArenaSim.startercards)),
        bonus_packs = Dict( :M19 => 0)));
summarize_results(res.packs_opened)

Dict{Symbol,Float64} with 8 entries:
  :DOM => 44.9174
  :XLN => 67.7534
  :HOU => 0.0
  :M19 => 0.0
  :RIX => 0.0
  :AER => 0.0171
  :AKH => 5.2624
  :KLD => 3.4642

"Total (10%, mean, 90%):"

(105.0, 121.4145, 138.0)

# Starting Account, no NPE
3 packs of M19, spend starting gold on best set from above results

In [6]:
res = simulate(10000, ArenaSim.deckinfo.(mtgadecks);
    parameters = ArenaSim.SimParameters(welcome_bundle = false,
        kaladesh_grant = false,starter_cards = zeros(Int, length(ArenaSim.startercards)),
        bonus_packs = Dict( :M19 => 3, :DOM => 5)));
summarize_results(res.packs_opened)

Dict{Symbol,Float64} with 8 entries:
  :DOM => 38.7019
  :XLN => 67.1487
  :HOU => 0.0
  :M19 => 0.0
  :RIX => 0.0
  :AER => 0.0068
  :AKH => 4.8477
  :KLD => 3.1865

"Total (10%, mean, 90%):"

(98.0, 113.8916, 130.0)

# Starting Account, no NPE, Welcome Bundle

In [7]:
res = simulate(10000, ArenaSim.deckinfo.(mtgadecks);
    parameters = ArenaSim.SimParameters(welcome_bundle = true,
        kaladesh_grant = false,starter_cards = zeros(Int, length(ArenaSim.startercards)),
        bonus_packs = Dict( :M19 => 3, :DOM => 5)));
summarize_results(res.packs_opened)

Dict{Symbol,Float64} with 8 entries:
  :DOM => 30.5498
  :XLN => 63.3124
  :HOU => 0.0
  :M19 => 0.0
  :RIX => 0.0
  :AER => 0.0007
  :AKH => 3.0064
  :KLD => 1.5958

"Total (10%, mean, 90%):"

(83.0, 98.4651, 113.0)

# Starting Account, NPE complete, Welcome Bundle
Not truly representative of someone who wants to buy in on literally day 1, as this takes a month to complete.  

In 4 weeks:

* 10 M19 packs from NPE
* 12 M19 packs from weekly quests
* 2500 gold from NPE
* 17850 gold from quest + 1 win on most days ( (600 + 250) * 21 )

Split the gold between DOM and XLN for 10 packs each relative to above scenario

In [8]:
res = simulate(10000, ArenaSim.deckinfo.(mtgadecks);
    parameters = ArenaSim.SimParameters(welcome_bundle = true,
        kaladesh_grant = false,starter_cards = ArenaSim.startercards,
        bonus_packs = Dict( :M19 => 25, :DOM => 15, :XLN => 10)));
summarize_results(res.packs_opened)

Dict{Symbol,Float64} with 8 entries:
  :DOM => 22.6141
  :XLN => 30.4176
  :HOU => 0.0
  :M19 => 0.0
  :RIX => 0.0
  :AER => 0.0016
  :AKH => 2.534
  :KLD => 1.1225

"Total (10%, mean, 90%):"

(41.0, 56.6898, 71.0)